In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact
import copy
from IPython.display import HTML,display

# What is happening in this notebook
A class is created to define a 'Cell'. This is an object that can move in four directions (up, down, left, right) across a lattice. Here cells are defined with a spatial location (x,y) and are considered points on a lattice. Initial distribution of cells across the lattice is random and depends on the percentage of lattice covered initially. They can self divide based on a poisson distribution. The average time between birth and cell division (an subsequent cell division) is defined as once every day or 24 hrs. The cells are simulated over a period of time to observe population growth dependent on initial conditions. 

Here a visual of the lattice at each time step t is created to insure cells are behaving as expected.

In [2]:
# %matplotlib inline
class Cell:
    '''
    Define the characteristics of a cell:
    initial conditions - position age etc
    '''
    
    def __init__(self, x, y, prob, mu, sigma):
        self.x = x
        self.y = y
        self.prob = prob
        if prob == 'poisson':
            self.division_time = int(np.round(np.random.poisson(mu))) # normal distribution of cell division time
        elif prob == 'normal':
            self.division_time = int(np.round(np.random.normal(mu, sigma))) # normal distribution of cell division time
        self.age = random.randint(0,self.division_time)

    def move(self, lattice):
        '''
        Describes how the cell moves on the lattice: cell can only move up,down left and right
        No two cells can be on the same lattice point
        '''
        direction = random.choice(['up', 'down', 'left', 'right'])
        
        if direction == 'up' and self.y + 1 < len(lattice) and not lattice[self.x, self.y + 1]:
            self.y += 1
        elif direction == 'down' and self.y - 1 >= 0 and not lattice[self.x, self.y - 1]:
            self.y -= 1
        elif direction == 'left' and self.x - 1 >= 0 and not lattice[self.x - 1, self.y]:
            self.x -= 1
        elif direction == 'right' and self.x + 1 < len(lattice) and not lattice[self.x + 1, self.y]:
            self.x += 1

    def divide(self, lattice):
        '''
        Describes how a cell divides.
        This function identifies the existing available lattice points that could be occupied by a new cell
        Only neighbouring cells (Von Neuman neighbourhood) are considered
        '''
        available_positions = []  # create a list of available lattice points
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_x, new_y = self.x + dx, self.y + dy
            # check that the new coordinates are within the boundaries of the lattice and are unoccupied
            if 0 <= new_x < len(lattice) and 0 <= new_y < len(lattice[0]) and not lattice[new_x, new_y]:
                available_positions.append((new_x, new_y))

        if available_positions:
            new_x, new_y = random.choice(available_positions)
            new_cell = Cell(new_x, new_y, self.prob, mu, sigma)
            lattice[new_x, new_y] = True
            return new_cell
        else:
            return None

def plot_cells(t, cells_list, lattice_size):
    '''
    Function to plot cells on a lattice square grid
    Sizes optimized for 10x10 but could be sized up
    '''
    plt.close('all') # Clear the previous plot
    current_cells = cells_list[t-1]  # Get the cells at the specified time step
    
    fig = plt.figure(figsize=(5, 5))

    for idx, cell in enumerate(current_cells):
        plt.scatter(cell.x, cell.y, label=f'Cell {idx + 1}', s=500, alpha=0.7)

    plt.title(f"Visualisation of the lattice at t={t} hrs")
    plt.grid(True, which='both', linewidth=1)
    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    plt.xlim(-1, lattice_size)
    plt.ylim(-1, lattice_size)
    plt.xticks(range(lattice_size))
    plt.yticks(range(lattice_size))
    plt.show()

def simulate_movement(num_cells, t, lattice_size,  prob,  mu, sigma):
    '''
    Function to simulate the movement of cells on the lattice
    Cells can move into available adjacent positions
    When a cell is old enough (self.division_time) a cell can divide into an availble lattice point
    If there are no available lattice points the cell does not divide    
    '''
    lattice = np.zeros((lattice_size, lattice_size), dtype=bool)
    unique_start_positions = random.sample(range(lattice_size * lattice_size), num_cells)
    cells_list = []  # List to store cells at each time step
    cells = [Cell(pos % lattice_size, pos // lattice_size, prob, mu, sigma) for pos in unique_start_positions]
    cells_list.append(copy.deepcopy(cells))

    for step in range(1, t+1):
        new_cells = []
        for cell in cells:
            lattice[cell.x, cell.y] = False
            cell.move(lattice)
            lattice[cell.x, cell.y] = True
            cell.age += 1

            if cell.age % cell.division_time == 0:
                new_cell = cell.divide(lattice)
                if new_cell:
                    new_cells.append(new_cell)
                    cells.append(new_cell)

        cells_list.append(copy.deepcopy(cells))

    return cells_list  


In [9]:

if __name__ == "__main__":
    t_max = 200 # simulate model up to this time point (hours)
    lattice_size = int(input("For a nxn lattice what is n?:")) # square lattice 
    mu = 24 # average time between cell division (hrs)
    sigma = 5 # standard dist. for cell division (hrs)
    prob = 'poisson' # what probability curve cell division is decide by, choose poisson/gaus etc
    perc = float(input("What percent of the lattice is filled?:"))  # how much is initial lattice occupied
    num_cells = int((lattice_size**2)*perc) # how many initial cells

    # run model
    final_cells_list = simulate_movement(num_cells, t_max, lattice_size, prob,  mu, sigma) 

        
    # Define custom widgets for interactive plot
    t_value = widgets.IntText(value=1, min=1, max=t_max, step=1, description='t:')
    interact(plot_cells, t=t_value, cells_list=widgets.fixed(final_cells_list), lattice_size=widgets.fixed(lattice_size));
    



For a nxn lattice what is n?: 10
What percent of the lattice is filled?: 0.1


interactive(children=(IntText(value=1, description='t:'), Output()), _dom_classes=('widget-interact',))

In [5]:
# %matplotlib inline
class Cell:
    '''
    Define the characteristics of a cell:
    initial conditions - position age etc
    '''
    
    def __init__(self, x, y, prob, mu, sigma):
        self.x = x
        self.y = y
        self.prob = prob
        if prob == 'poisson':
            self.division_time = int(np.round(np.random.poisson(mu))) # normal distribution of cell division time
        elif prob == 'normal':
            self.division_time = int(np.round(np.random.normal(mu, sigma))) # normal distribution of cell division time
        self.age = random.randint(0,self.division_time)
        
    def move(self, lattice):
        '''
        Describes how the cell moves on the lattice: cell can only move up, down, left, and right
        No two cells can be on the same lattice point
        '''
        direction = random.choice(['up', 'down', 'left', 'right'])
    
        # Set the current lattice point as unoccupied before moving
        
        if direction == 'up' and self.y + 1 < len(lattice) and not lattice[self.x, self.y + 1]:
            self.y += 1
        elif direction == 'down' and self.y - 1 >= 0 and not lattice[self.x, self.y - 1]:
            self.y -= 1
        elif direction == 'left' and self.x - 1 >= 0 and not lattice[self.x - 1, self.y]:
            self.x -= 1
        elif direction == 'right' and self.x + 1 < len(lattice) and not lattice[self.x + 1, self.y]:
            self.x += 1
    
        # Set the new lattice point as occupied after moving
        
    def divide(self, lattice):
        '''
        Describes how a cell divides.
        This function identifies the existing available lattice points that could be occupied by a new cell
        Only neighbouring cells (Von Neuman neighbourhood) are considered
        '''
        available_positions = []  # create a list of available lattice points
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_x, new_y = self.x + dx, self.y + dy
            # check that the new coordinates are within the boundaries of the lattice and are unoccupied
            if 0 <= new_x < len(lattice) and 0 <= new_y < len(lattice[0]) and not lattice[new_x, new_y]:
                available_positions.append((new_x, new_y))

        if available_positions:
            new_x, new_y = random.choice(available_positions)
            new_cell = Cell(new_x, new_y, self.prob, mu, sigma)
            lattice[new_x, new_y] = True
            return new_cell
        else:
            return None

def plot_cells(t, cells_list, lattice_size):
    '''
    Function to plot cells on a lattice square grid
    Sizes optimized for 10x10 but could be sized up
    '''
    plt.close('all') # Clear the previous plot
    current_cells = cells_list[t - 1]  # Get the cells at the specified time step
    
    plt.figure(figsize=(5, 5))

    for idx, cell in enumerate(current_cells):
        plt.scatter(cell.x, cell.y, label=f'Cell {idx + 1}', s=500, alpha=0.7)

    plt.title(f"Visualisation of the lattice at t={t} hrs")
    plt.grid(True, which='both', linewidth=1)
    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    plt.xlim(-1, lattice_size)
    plt.ylim(-1, lattice_size)
    plt.xticks(range(lattice_size))
    plt.yticks(range(lattice_size))
    plt.show()

def simulate_movement(num_cells, t, lattice_size,  prob,  mu, sigma):
    '''
    Function to simulate the movement of cells on the lattice
    Cells can move into available adjacent positions
    When a cell is old enough (self.division_time) a cell can divide into an availble lattice point
    If there are no available lattice points the cell does not divide    
    '''
    lattice = np.zeros((lattice_size, lattice_size), dtype=bool)
    unique_start_positions = random.sample(range(lattice_size * lattice_size), num_cells)
    cells_list = []  # List to store cells at each time step
    cells = [Cell(pos % lattice_size, pos // lattice_size, prob, mu, sigma) for pos in unique_start_positions]
    cells_list.append(copy.deepcopy(cells))
    
    for cell in cells[:]:
        lattice[cell.x,cell.y] = True
    
    for step in range(0, t+1):
        new_cells = []
        for cell in cells:
            lattice[cell.x, cell.y] = False
            cell.move(lattice)
            lattice[cell.x, cell.y] = True
            cell.age += 1

            if cell.age % cell.division_time == 0:
                new_cell = cell.divide(lattice)
                if new_cell:
                    new_cells.append(new_cell)
                    cells.append(new_cell)

        cells_list.append(copy.deepcopy(cells))

    return cells_list  


In [8]:

if __name__ == "__main__":
    t_max = 200
    num_simulations = 5
    lattice_size = int(input("For a nxn lattice what is n?:"))
    mu = 24
    sigma = 5
    prob = 'poisson'
    perc = float(input("What percent of the lattice is filled?:"))
    num_cells = int((lattice_size**2)*perc) 
    final_cells_list = simulate_movement(num_cells, t_max, lattice_size, prob,  mu, sigma)

        
    # Define custom widgets for interactive plot
    t_value = widgets.IntText(value=1, min=1, max=t_max, step=1, description='t:')
    interact(plot_cells, t=t_value, cells_list=widgets.fixed(final_cells_list), lattice_size=widgets.fixed(lattice_size));
    



For a nxn lattice what is n?: 10
What percent of the lattice is filled?: 0.1


interactive(children=(IntText(value=1, description='t:'), Output()), _dom_classes=('widget-interact',))